In [1]:
import sys
sys.path.append("../")
from pattern_extraction import *
from model_unit_series import field_lookup
import sc2reader

In [2]:
import pickle
import numpy as np
from itertools import groupby
from functools import partial
import matplotlib.pyplot as plt

In [3]:
results_path = "results/small"
race = "Terran"

In [4]:
with open("valid_game_ids.txt") as fp:
    replay_files = [x.strip() for x in fp.readlines() if x.startswith("spawningtool")]

In [5]:
with open(f"{results_path}/race_lookup.pickle", "rb") as fp:
    race_lookup = pickle.load(fp)

In [6]:
all_series = np.loadtxt(f"{results_path}/{race}/all_series.txt")

In [7]:
krange = range(3, 21)
sub_krange = range(2, 11)
cluster_lookup, mrf_lookup, model_lookup, bic_lookup = load_TICC_output(results_path, [race], krange)

In [8]:
results_path_race = f"{results_path}/{race}"
with open(f"{results_path_race}/idx_lookup.pickle", 'rb') as fp:
    idx_lookup = pickle.load(fp)
noise = np.array([-1] * len(field_lookup[race]))
subseries_lookups = {}
cd = cluster_lookup[race]
md = mrf_lookup[race]
ser = all_series
no_dups = {k for k in cd if
           not any(any((m == md[k][ci]).all() for cj, m in md[k].items() if cj > ci) for ci in range(k))}
print(f"{race} k with no dups: {no_dups}")
null_clusters = {k: [ci for ci in md[k] if is_null_cluster(md[k][ci])] for k in md}
handles_noise = {k for k in cd if len(null_clusters[k]) > 0 and all(
    cd[k][i] in null_clusters[k] for i in range(len(ser)) if (ser[i] == noise).all())}
print(f"{race} k that handle noise: {handles_noise}")
valid_k = no_dups.intersection(handles_noise)
for k in valid_k:
    patterns = get_patterns(mrf_lookup[race][k], cluster_lookup[race][k], idx_lookup)
    subseries_lookups[k] = make_subseries_lookup(k, patterns, mrf_lookup[race][k], all_series, noise)

Terran k with no dups: {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
Terran k that handle noise: {4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}


In [9]:
sub_lookup = load_sub_lookup(results_path_race, subseries_lookups, sub_krange)
sub_clusters = sub_lookup.clusters
pattern_lookup = get_pattern_lookups(valid_k, sub_clusters, sub_lookup.mrfs, subseries_lookups,
                                     cluster_lookup[race], mrf_lookup[race], idx_lookup)

In [11]:
metas = {}
for rf in replay_files:
    with open(f"replays/{rf.replace('.SC2Replay', '_meta.json')}") as fp:
        meta = json.load(fp)
        metas[rf.split("_")[1].split(".")[0]] = meta

In [ ]:
matchups = []
rng = np.random.default_rng(42)
for rf in replay_files:
    with open(f"replays/{rf.replace('.SC2Replay', '_meta.json')}")as fp:
        matchups.extend(metas[rf.split("_")[1].split(".")[0]]['tags'].get("Matchup", ["Missing"]))
matchup_frac = {m: c / sum(matchups_count.values()) for m, c in matchups_count.items()}
weights = np.array([matchup_frac[m["tags"]["Matchup"][0]] for m in metas.values()])
sample_gids = rng.choice(list(metas.keys()), args.sample, False, weights / weights.sum())
sample_replay_files = [f"spawningtool_{x}.SC2Replay" for x in gids]



In [10]:
ids = []
for uid, gid, _ in idx_lookup:
    r = sc2reader.load_replay(f"replays/spawningtool_{gid}.SC2Replay")
    p = next(p for p in r.players if p.detail_data['bnet']['uid'] == uid)
    assert p.play_race == race
    ids.append((uid, gid, p.name))     

In [47]:
assert all(os.path.exists(f"replays/spawningtool_{gid}_meta.json") for _, gid, _ in ids)
uid_to_name = {}
for uid, gid, name in ids:
#     print(name, metas[gid].keys())
    assert name in metas[gid] or name in [x['alias'] for x in metas[gid].values() if 'alias' in x]
    assert len(metas[gid]) == 3
    realname = name if name in metas[gid] else next(n for n, x in metas[gid].items() if "alias" in x and name == x["alias"])
    assert uid not in uid_to_name or any(n.lower() == realname.lower() for n in uid_to_name[uid]), (uid, realname, uid_to_name[uid])
    uid_to_name.setdefault(uid, set()).add(realname)
uid_to_name

{1: {'uThermal'},
 11: {'Clem'},
 13: {'soul'},
 21: {'Optimus'},
 26: {'Lillekanin'},
 27: {'InZaNe'},
 31: {'Okaneby'},
 82: {'Maru'},
 83: {'DeMusliM'},
 88: {'MaSa'},
 98: {'lucifron'},
 111: {'kauP'},
 145: {'SpeCial'},
 155: {'Kelazhur'},
 268: {'Buster'},
 288: {'HeroMarine'},
 306: {'Future'},
 307: {'Cuddlebear'},
 308: {'Kozan'},
 325: {'soul'},
 334: {'Jason'},
 778: {'uThermal'},
 784: {'MaSa'},
 787: {'SpeCial'},
 797: {'Ziggy'},
 804: {'Optimus'},
 819: {'DeMusliM'},
 825: {'Lillekanin'},
 826: {'kauP'},
 831: {'erL'},
 847: {'HeRoMaRinE'},
 855: {'BlueCheese'},
 882: {'Kas'},
 995: {'Lillekanin'},
 997: {'Ryung'},
 998: {'souL'},
 999: {'TY'},
 1024: {'Dayshi'},
 1028: {'SpeCial'},
 1029: {'Ziggy'},
 1040: {'BlueCheese'},
 1636: {'SpeCial'},
 273496: {'Lillekanin'},
 284466: {'DeMusliM'},
 315071: {'Serral'},
 327563: {'avilo'},
 383770: {'Vindicta'},
 622096: {'TaeJa'},
 635980: {'DerTryharder'},
 809005: {'AqueroN'},
 810627: {'Jason'},
 1029628: {'InZaNe'},
 1143713: 

In [31]:
with open(f"{results_path}/race_lookup.pickle", "rb") as fp:
    race_lookup = pickle.load(fp)

In [33]:
sum([len(race_lookup[race]) for race in field_lookup])

1600

In [13]:
any(p1 == p2 for p1, p2 in [[x for x in m if x != "tags"] for m in metas.values()])

False

In [14]:
[(gid, m) for gid, m in metas.items() if len([x for x in m if x != "tags"]) != 2]

[]

In [56]:
gids = {gid for _, gid, _ in ids}
opening_lookup = {o: [] for gid, m in metas.items() if gid in gids for v in m.values() if "Opening" in v.get("tags", {}) for o in v["tags"]["Opening"]}
for uid, gid, _ in ids:
    meta = metas[gid]
    name = next(n for n in uid_to_name[uid] if n in meta)
    tags = meta[name]["tags"]
    if "Opening" in tags:
        for o in tags["Opening"]:
            opening_lookup[o].append((uid, gid))
opening_lookup = {k: v for k, v in opening_lookup.items() if len(v) > 0}
opening_lookup

{'Reaper Expand': [(1, '39874'),
  (11, '39573'),
  (13, '39939'),
  (21, '39921'),
  (27, '39707'),
  (82, '34799'),
  (82, '34802'),
  (83, '39872'),
  (111, '39703'),
  (145, '39832'),
  (155, '39560'),
  (288, '40687'),
  (306, '40722'),
  (308, '40709'),
  (325, '40602'),
  (334, '40706'),
  (778, '34027'),
  (804, '33895'),
  (804, '34129'),
  (804, '34133'),
  (819, '33844'),
  (825, '33818'),
  (825, '34188'),
  (826, '33942'),
  (826, '34051'),
  (826, '34139'),
  (831, '33886'),
  (847, '33799'),
  (855, '33963'),
  (882, '33875'),
  (995, '34812'),
  (997, '34832'),
  (998, '34825'),
  (1028, '34992'),
  (1040, '35000'),
  (1636, '34406'),
  (1636, '35122'),
  (284466, '38923'),
  (284466, '38924'),
  (315071, '42632'),
  (315071, '42635'),
  (383770, '33482'),
  (383770, '48274'),
  (622096, '56155'),
  (810627, '34601'),
  (810627, '37800'),
  (1029628, '39117'),
  (1029628, '39121'),
  (1029628, '39136'),
  (1029628, '39163'),
  (1029628, '45516'),
  (1029628, '49435'),
 

In [76]:
yes = u'\u2714'
no = u'\u2718'
for k in pattern_lookup:
    print(f"\nk = {k}")
    opening_patterns = [p for p in pattern_lookup[k]["base"] if p.start_idx == idx_lookup[(p.uid, p.pid, race)][0]]
    pcounts = Counter([p.cid for p in opening_patterns])
    for opening, pairs in opening_lookup.items():
        ps = [p for p in opening_patterns if (p.uid, p.pid) in pairs]
        other_ps = [p for p in opening_patterns if (p.uid, p.pid) not in pairs]
        assert len(ps) == len(pairs), (len(ps), len(pairs))
        consistent = all(p.cid == ps[0].cid for p in ps)
        distinctive = any(all(p.cid != q.cid for q in other_ps) for p in ps)
        
        if consistent:
            print(opening)
            cid = ps[0].cid
            for sub_k in pattern_lookup[k][cid]:
                ops = [p for p in pattern_lookup[k][cid][sub_k] if p.start_idx == idx_lookup[(p.uid, p.pid, race)][0]]
                ps = [p for p in ops if (p.uid, p.pid) in pairs]
                other_ps = [p for p in ops if (p.uid, p.pid) not in pairs]
#                 assert len(ps) == len(pairs), (len(ps), len(pairs))
                consistent = all(p.cid == ps[0].cid for p in ps)
                distinctive = any(all(p.cid != q.cid for q in other_ps) for p in ps)
                coverage = len(ps) == len(pairs)
                print("    {:25}  consistent {}  distinctive {}  coverage {}".format(f"{k}[{cid}], {sub_k}", 
                                                                                     yes if consistent else no, 
                                                                                     yes if distinctive else no,
                                                                                     yes if coverage else no))
        else:
            print(f"{opening:25}  consistent {yes if consistent else no}  distinctive {yes if distinctive else no}")
    # for each opening
        # consistent if all start patterns are the same
        # distinctive is no other patterns


k = 4
Reaper Expand
    4[0], 0                    consistent ✔  distinctive ✘  coverage ✔
    4[0], 3                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 4                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 5                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 6                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 7                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 8                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 9                    consistent ✔  distinctive ✔  coverage ✘
    4[0], 10                   consistent ✔  distinctive ✔  coverage ✘
Liberator Opening
    4[0], 0                    consistent ✔  distinctive ✘  coverage ✔
    4[0], 3                    consistent ✔  distinctive ✘  coverage ✘
    4[0], 4                    consistent ✔  distinctive ✘  coverage ✘
    4[0], 5                    consistent ✔  distinctive ✘  coverage ✘
    4[0], 6                    consist

Hellion Opening
    14[0], 0                   consistent ✔  distinctive ✘  coverage ✔
    14[0], 2                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 3                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 4                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 5                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 6                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 7                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 8                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 9                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 10                  consistent ✔  distinctive ✘  coverage ✘
1/1/1
    14[0], 0                   consistent ✔  distinctive ✘  coverage ✔
    14[0], 2                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 3                   consistent ✔  distinctive ✘  coverage ✘
    14[0], 4                   consistent ✔  distinctiv

In [54]:
[m for gid, m in metas.items() if gid in gids]

[{'tags': {'Map': ['Deathaura LE'],
   'Matchup': ['TvZ'],
   'Race': ['Terran', 'Zerg'],
   'Spawn Locations': ['Cross Positions']},
  'Kelazhur': {'tags': {'Composition': ['Hellion Banshee'],
    'League': ['Master'],
    'Opening': ['Reaper Expand']},
   'winner': True,
   'name': 'Kelazhur',
   'league': 'Master'},
  'Cham': {'tags': {'League': ['Master'], 'Opening': ['Hatch First']},
   'winner': False,
   'name': 'Cham',
   'league': 'Master'}},
 {'tags': {'Event': ['The Miserable Cup'],
   'Map': ['Ice and Chrome LE'],
   'Matchup': ['PvT'],
   'Race': ['Protoss', 'Terran'],
   'Replay Pack': ['The Miserable Cup Replay Pack'],
   'Spawn Locations': ['Cross Positions']},
  'Creature': {'tags': {'Composition': ['Blink Stalkers'],
    'Opening': ['1 Gate Expand']},
   'winner': False,
   'name': 'Creature',
   'league': None},
  'Jason': {'tags': {'League': ['Grandmaster']},
   'winner': True,
   'name': 'Jason',
   'league': 'Grandmaster'}},
 {'tags': {'Event': ['The Miserable Cup

In [65]:
for k in pattern_lookup:
    for cid in {p.cid for p in pattern_lookups[k]["base"]}:
        

dict_values([(0, 102), (202, 314), (414, 568), (668, 761), (861, 960), (1060, 1170), (1270, 1356), (1456, 1552), (1652, 1776), (1876, 1925), (2025, 2081), (2181, 2296), (2396, 2511), (2611, 2686), (2786, 2888), (2988, 3141), (3241, 3341), (3441, 3576), (3676, 3728), (3828, 3888), (3988, 4065), (4165, 4213), (4313, 4412), (4512, 4590), (4690, 4741), (4841, 4905), (5005, 5117), (5217, 5263), (5363, 5427), (5527, 5634), (5734, 5862), (5962, 6056), (6156, 6285), (6385, 6468), (6568, 6615), (6715, 6783), (6883, 6983), (7083, 7260), (7360, 7449), (7549, 7638), (7738, 7999), (8099, 8549), (8649, 8730), (8830, 8937), (9037, 9089), (9189, 9259), (9359, 9450), (9550, 9638), (9738, 9843), (9943, 9997), (10097, 10153), (10253, 10345), (10445, 10508), (10608, 10713), (10813, 10934), (11034, 11088), (11188, 11366), (11466, 11589), (11689, 11748), (11848, 11895), (11995, 12076), (12176, 12288), (12388, 12487), (12587, 12691), (12791, 12925), (13025, 13116), (13216, 13329), (13429, 13511), (13611, 137